In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras import layers
from keras.models import Sequential

MAX_SEQUENCE_LENGTH = 5000
MAX_NUM_WORDS = 25000
EMBEDDING_DIM = 300
TEST_SPLIT = 0.2

TEXT_DATA = '/content/fake_or_real_news.csv'

df = pd.read_csv(TEXT_DATA)
df.drop(labels=['id','title'], axis='columns', inplace=True)
mask = list(df['text'].apply(lambda x: len(x) > 0))
df = df[mask]
texts = df['text']
labels = df['label']



def evaluate_model(predict_fun, X_train, y_train, X_test, y_test):
    '''
    evaluate the model, both training and testing errors are reported
    '''
    # training error
    y_predict_train = predict_fun(X_train)
    train_acc = accuracy_score(y_train,y_predict_train)

    # testing error
    y_predict_test = predict_fun(X_test)
    test_acc = accuracy_score(y_test,y_predict_test)

    return train_acc, y_predict_train

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
data = pad_sequences(sequences,
                     maxlen=MAX_SEQUENCE_LENGTH,
                     padding='pre',
                     truncating='pre')

x_train, x_val, y_train, y_val = train_test_split(data, labels.apply(lambda x: 0 if x == 'FAKE' else 1),
                                                  test_size=TEST_SPLIT)


model = Sequential(
    [

        layers.Embedding(num_words,
                         EMBEDDING_DIM,
                         input_length=MAX_SEQUENCE_LENGTH,
                         trainable=True),
        layers.Conv1D(128, 5, activation='relu'),
        layers.GlobalMaxPooling1D(),

        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

history = model.fit(x_train,
                    y_train,
                    batch_size=128,
                    epochs=2,
                    validation_data=(x_val, y_val))
train_acc, y_predict_train = evaluate_model(lambda x: np.rint(model.predict(x)),
                                     x_train,
                                     y_train,
                                     x_val,
                                     y_val)

print("Training Accuracy: {:.2f}%".format(train_acc*100))
#print("Testing Accuracy: {:.2f}%".format(test_acc*100))
print(y_predict_train)

Epoch 1/2
40/40 [==============================] - 858s 21s/step - loss: 0.6289 - accuracy: 0.6602 - val_loss: 0.4792 - val_accuracy: 0.8090
Epoch 2/2
40/40 [==============================] - 55s 1s/step
Training Accuracy: 93.73%
[[1.]
 [0.]
 [1.]
 ...
 [1.]
 [0.]
 [0.]]
